In [0]:
"""Notebook MVP 00 - Gerar tabela Bronze (ciclos_concatenados).

Este notebook le e concatena os arquivos Excel Ciclo01.xlsx e Ciclo02.xlsx
e gera o arquivo Parquet Bronze no diretorio mvp/dados_bronze.

Toda a logica esta incorporada diretamente, sem necessidade de importar modulos externos.
"""

import pandas as pd
import numpy as np
import os
from pyspark.sql import SparkSession


print("=== PARTE 1: CARREGANDO ARQUIVOS EXCEL ===\n")

caminhos_possiveis = [
    "/Workspace/Users/romulobrtsilva@gmail.com/Drafts/raw_data",
]
    # caminho_ciclo01 =  "/Workspace/Users/romulobrtsilva@gmail.com/Drafts/raw_data/Ciclo01.xlsx"
    # caminho_ciclo02 = "/Workspace/Users/romulobrtsilva@gmail.com/Drafts/raw_data/Ciclo02.xlsx"
caminho_ciclo01 = None
caminho_ciclo02 = None

for caminho_base in caminhos_possiveis:
    arquivo1 = os.path.join(caminho_base, "Ciclo01.xlsx")
    arquivo2 = os.path.join(caminho_base, "Ciclo02.xlsx")
    if os.path.exists(arquivo1) and os.path.exists(arquivo2):
        caminho_ciclo01 = arquivo1
        caminho_ciclo02 = arquivo2
        print(f"[OK] Arquivos encontrados em: {caminho_base}")
        break

if caminho_ciclo01 is None or caminho_ciclo02 is None:
    raise FileNotFoundError(
        "Arquivos Excel (Ciclo01.xlsx e Ciclo02.xlsx) nao encontrados.\n"
        "Verifique se os arquivos estao em uma das seguintes localizacoes:\n"
        "  - /dbfs/Workspace/Users/romulobrtsilva@gmail.com/Drafts/raw_data\n"
        "  - /dbfs/FileStore/shared_uploads\n"
        "  - /dbfs/mnt/mvp/dados_bronze"
    )

print(f"\nCaminhos configurados:")
print(f"  Ciclo01.xlsx: {caminho_ciclo01}")
print(f"  Ciclo02.xlsx: {caminho_ciclo02}")

# Instalar openpyxl se necessario
try:
    import openpyxl
    print("\n[OK] openpyxl ja instalado")
except ImportError:
    print("\nInstalando openpyxl...")
    %pip install openpyxl
    import openpyxl

# Carregar arquivos Excel
print("\nCarregando Ciclo01.xlsx...")
df_ciclo01 = pd.read_excel(caminho_ciclo01)
print(f"Ciclo01.xlsx: {len(df_ciclo01):,} linhas")

print("Carregando Ciclo02.xlsx...")
df_ciclo02 = pd.read_excel(caminho_ciclo02)
print(f"Ciclo02.xlsx: {len(df_ciclo02):,} linhas")

# Concatenar os DataFrames
print("\nConcatenando DataFrames...")
df_concatenado = pd.concat([df_ciclo01, df_ciclo02], ignore_index=True)

print(f"\nDataFrame concatenado criado:")
print(f"  Total de linhas: {len(df_concatenado):,}")
print(f"  Total de colunas: {len(df_concatenado.columns)}")
print(f"  Colunas disponiveis: {list(df_concatenado.columns)}")

# ============================================================================
# Validacao dos dados
# ============================================================================

print("\n=== PARTE 2: VALIDACAO DOS DADOS ===\n")

print("Primeiras 5 linhas:")
print(df_concatenado.head())

print("\nInformacoes do DataFrame:")
print(df_concatenado.info())



=== PARTE 1: CARREGANDO ARQUIVOS EXCEL ===

[OK] Arquivos encontrados em: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/raw_data

Caminhos configurados:
  Ciclo01.xlsx: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/raw_data/Ciclo01.xlsx
  Ciclo02.xlsx: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/raw_data/Ciclo02.xlsx

[OK] openpyxl ja instalado

Carregando Ciclo01.xlsx...
Ciclo01.xlsx: 752,223 linhas
Carregando Ciclo02.xlsx...
Ciclo02.xlsx: 796,343 linhas

Concatenando DataFrames...

DataFrame concatenado criado:
  Total de linhas: 1,548,566
  Total de colunas: 9
  Colunas disponiveis: ['DataEv', 'HoraEv', 'CodRecurso', 'Recursos', 'Latitude', 'Longitude', 'SiglaRodovia', 'Sentido', 'KM']

=== PARTE 2: VALIDACAO DOS DADOS ===

Primeiras 5 linhas:
      DataEv    HoraEv  CodRecurso  ... SiglaRodovia  Sentido       KM
0 2025-05-01  00:00:05         135  ...    BR-153/GO        S  212+768
1 2025-05-01  00:00:07         134  ...    BR-153/GO        N  367+530
2 2025-05-01  00:00

In [0]:
# ============================================================================
# Configuracao de caminhos de saida 
# ============================================================================

print("\n=== PARTE 3: SALVANDO DADOS BRONZE ===\n")

# CORRECAO: Usar caminho absoluto correto (com / no inicio)
WORKSPACE_BASE = "/Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp"
BRONZE_DIR = f"{WORKSPACE_BASE}/dados_bronze"  # SEM duplicacao

# Criar diretorio se nao existir
os.makedirs(BRONZE_DIR, exist_ok=True)
print(f"[OK] Diretorio criado/verificado: {BRONZE_DIR}")

# Caminho completo do arquivo de saida
ARQ_BRONZE = f"{BRONZE_DIR}/ciclos_concatenados.parquet"

print(f"Diretorio Bronze: {BRONZE_DIR}")
print(f"Arquivo Bronze de saida: {ARQ_BRONZE}")

# Salvar em Parquet
print("\nSalvando DataFrame concatenado em Parquet (Bronze)...")
df_concatenado.to_parquet(ARQ_BRONZE, index=False)
print(f"[OK] Arquivo Parquet salvo em: {ARQ_BRONZE}")

# Verificar se arquivo foi criado
if os.path.exists(ARQ_BRONZE):
    tamanho_mb = os.path.getsize(ARQ_BRONZE) / (1024 * 1024)
    print(f"[OK] Arquivo criado com sucesso! Tamanho: {tamanho_mb:.2f} MB")
else:
    print("[ERRO] Arquivo nao foi criado!")


=== PARTE 3: SALVANDO DADOS BRONZE ===

[OK] Diretorio criado/verificado: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze
Diretorio Bronze: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze
Arquivo Bronze de saida: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze/ciclos_concatenados.parquet

Salvando DataFrame concatenado em Parquet (Bronze)...
[OK] Arquivo Parquet salvo em: /Workspace/Users/romulobrtsilva@gmail.com/Drafts/mvp/dados_bronze/ciclos_concatenados.parquet
[OK] Arquivo criado com sucesso! Tamanho: 23.99 MB
